In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Model
from tensorflow.keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
import tensorflow as tf

In [2]:
X_train = load('trainX1.npy')
X_test = load('testX1.npy')
y_train = load('trainY1.npy')
y_test = load('testY1.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [3]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [4]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [5]:
model1 = Sequential()
model1.add(LSTM(512, input_shape=(n_timesteps,n_features)))
model1.add(Dense(256, activation='relu'))
model1.add(Dropout(0.1))
model1.add(Dense(128, activation='relu'))
model1.add(Dense(n_outputs, activation='softmax'))
model1.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model1.summary()
callbacks = [tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 512)               1062912   
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 118)               15222     
Total params: 1,242,358
Trainable params: 1,242,358
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model1.fit(X_training, y_training, epochs=200, verbose=True, validation_data=(X_validation, y_validation), batch_size=128,callbacks=callbacks)

Epoch 1/200
207/207 [==============================] - 1161s 6s/step - loss: 0.0075 - accuracy: 0.1925 - val_loss: 0.0052 - val_accuracy: 0.5165
Epoch 2/200
207/207 [==============================] - 1136s 5s/step - loss: 0.0032 - accuracy: 0.7299 - val_loss: 0.0019 - val_accuracy: 0.8429
Epoch 3/200
207/207 [==============================] - 1136s 5s/step - loss: 0.0017 - accuracy: 0.8572 - val_loss: 0.0016 - val_accuracy: 0.8660
Epoch 4/200
207/207 [==============================] - 1141s 6s/step - loss: 0.0013 - accuracy: 0.8902 - val_loss: 0.0012 - val_accuracy: 0.9015
Epoch 5/200
207/207 [==============================] - 1140s 6s/step - loss: 0.0011 - accuracy: 0.9098 - val_loss: 0.0010 - val_accuracy: 0.9162
Epoch 6/200
207/207 [==============================] - 1152s 6s/step - loss: 9.2084e-04 - accuracy: 0.9234 - val_loss: 0.0010 - val_accuracy: 0.9156
Epoch 7/200
207/207 [==============================] - 1148s 6s/step - loss: 7.9690e-04 - accuracy: 0.9339 - val_loss: 7.9029e

In [7]:
loss, accuracy = model1.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model1.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9607
Testing Accuracy: 0.8995
